In [4]:
import pandas as pd
import mysql.connector
import datetime

import json

In [5]:
host = 'localhost'
user = 'root'
password = ''
database = 'facultyleavedb'

In [6]:
def with_permission_fun(from_prefix, to_prefix, from_suffix,to_suffix):
    prefix = []
    if from_prefix and to_prefix:
        prefix.append(f"{from_prefix.strftime('%d-%m-%Y')} to {to_prefix.strftime('%d-%m-%Y')}")
    elif not from_prefix and to_prefix:
        prefix.append(f"{to_prefix.strftime('%d-%m-%Y')}")
    elif from_prefix and not to_prefix:
        prefix.append(f"{from_prefix.strftime('%d-%m-%Y')}")
    
    suffix = []
    if from_suffix and to_suffix:
        suffix.append(f"{from_suffix.strftime('%d-%m-%Y')} to {to_suffix.strftime('%d-%m-%Y')}")
    elif not from_suffix and to_suffix:
        suffix.append(f"{to_suffix.strftime('%d-%m-%Y')}")
    elif from_suffix and not to_suffix:
        suffix.append(f"{from_suffix.strftime('%d-%m-%Y')}")
    
    on_permission = []
    if prefix and suffix:
        on_permission.append(f"{prefix[0]} & {suffix[0]}")
    elif not prefix and suffix:
        on_permission.append(suffix[0])
    elif prefix and not suffix:
        on_permission.append(prefix[0])
    
    return on_permission


In [7]:
conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
cursor = conn.cursor()

In [8]:
query = "SELECT * FROM `staff` WHERE id=22015;"
cursor.execute(query)
data = cursor.fetchall()


In [7]:
cursor.reset()
on_permission  =[]
query = "SELECT `from`, `to`, `from_prefix`, `to_prefix`, `from_suffix`, `to_suffix`, `date_of_rejoin`, `total` FROM `el` WHERE id=22015; "
cursor.execute(query)
data2 = cursor.fetchall()
data2
data_el = [sublist + ("el",) for sublist in data2]
data_el
for i in data_el:
    on_permission.append(with_permission_fun(i[2],i[3],i[4],i[5]))

In [32]:
on_permission

[['2015-05-01 to 2015-05-03 & 2015-05-09 to 2015-05-10'],
 ['2015-07-12'],
 ['2015-08-08 to 2015-08-09 & 2015-08-15 to 2015-08-16'],
 ['2015-09-27 & 2015-10-02 to 2015-10-04'],
 [],
 [],
 []]

In [33]:
data_el_updated = []
for i,sublist in enumerate(data_el):
    # Extract the columns you want to keep
    columns_to_keep = sublist[:2] + (on_permission[i],) + sublist[6:]
    data_el_updated.append(columns_to_keep)

In [34]:
data_el_updated[0]

(datetime.date(2015, 5, 4),
 datetime.date(2015, 5, 8),
 ['2015-05-01 to 2015-05-03 & 2015-05-09 to 2015-05-10'],
 datetime.date(2015, 5, 11),
 5,
 'el')

In [35]:
cursor.reset()
query = "SELECT `from`, `to`, `medical_fittness_on`, `with_permission_on`, `doj`, `total` FROM `lop` WHERE id=22009; "
cursor.execute(query)
data3 = cursor.fetchall()
data_lop = [sublist + ("lop",) for sublist in data3]

ProgrammingError: 1054 (42S22): Unknown column 'with_permission_on' in 'field list'

In [89]:
cursor.reset()
query = "SELECT `from`, `to`, `medical_fittness_on`, `with_permission_on`, `doj`, `total` FROM `ml` WHERE id=22015; "
cursor.execute(query)
data4 = cursor.fetchall()
data_ml = [sublist + ("ml",) for sublist in data4]

In [90]:
cursor.reset()
query = "SELECT `from`, `to`, `medical_fittness_on`, `with_permission_on`, `doj`, `total` FROM `mtl` WHERE id=22015; "
cursor.execute(query)
data5 = cursor.fetchall()
data_mtl = [sublist + ("mtl",) for sublist in data5]

In [27]:
sorted_data = sorted(data_lop+data_el+data_ml+data_mtl, key=lambda x: x[0])
for item in sorted_data:
    print(item)

(datetime.date(2015, 5, 4), datetime.date(2015, 5, 8), ['01-05-2015 to 03-05-2015 & 09-05-2015 to 10-05-2015'], datetime.date(2015, 5, 11), 5, 'el')
(datetime.date(2015, 7, 7), datetime.date(2015, 7, 11), ['12-07-2015'], datetime.date(2015, 7, 13), 5, 'el')
(datetime.date(2015, 8, 10), datetime.date(2015, 8, 14), ['08-08-2015 to 09-08-2015 & 15-08-2015 to 16-08-2015'], datetime.date(2015, 8, 17), 5, 'el')
(datetime.date(2015, 9, 28), datetime.date(2015, 10, 1), ['27-09-2015 & 02-10-2015 to 04-10-2015'], datetime.date(2015, 10, 5), 4, 'el')
(datetime.date(2020, 3, 11), datetime.date(2020, 3, 14), [], None, 4, 'el')
(datetime.date(2020, 5, 19), datetime.date(2020, 5, 23), [], None, 5, 'el')
(datetime.date(2020, 11, 5), datetime.date(2020, 11, 11), [], datetime.date(2020, 11, 12), 7, 'el')
(datetime.date(2022, 3, 22), datetime.date(2022, 3, 27), 6, b'["28.03.222"]', [], datetime.date(2022, 3, 28), 'ml')
(datetime.date(2022, 5, 4), datetime.date(2022, 5, 18), 15, b'["2022-05-19"]', ['03-05

In [88]:
import xlsxwriter
workbook = xlsxwriter.Workbook('Book.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column('A:M', 12) 

need_remark = False

merge_ranges = [
    (2, 0, 2, 3),  
    (2, 4, 2, 12), 
    (3, 0, 3, 3),  
    (3, 4, 3, 12), 
    (4, 0, 4, 3),  
    (4, 4, 4, 12), 
    (5, 0, 5, 3),  
    (5, 4, 5, 12), 
    (6, 0, 6, 3),  
    (6, 4, 6, 12), 
    (7, 0, 7, 12),  
    (8, 0, 8, 4),
    (8, 5, 8, 10)
    
]
cell_format_center = workbook.add_format({
    'align': 'center',  # Center alignment
    'text_wrap': True,  # Wrap text
    
})
cell_format_left = workbook.add_format({
    'align': 'center',  # Center alignment
    'text_wrap': True,  # Wrap text
    
})


# Perform the defined merges
worksheet.merge_range(*(0,0,0,12), 'Annexure -II'	, cell_format_center)
worksheet.merge_range(*(1,0,1,12), 'Anna univeristy chennai', cell_format_center)
worksheet.merge_range(*merge_ranges[0], 'Name', cell_format_center)
worksheet.merge_range(*merge_ranges[1], data[0][1],cell_format_left)
worksheet.merge_range(*merge_ranges[2], 'Designation',cell_format_left)
worksheet.merge_range(*merge_ranges[3], '',cell_format_left)
worksheet.merge_range(*merge_ranges[4], 'Department',cell_format_left)
worksheet.merge_range(*merge_ranges[5], data[0][2],cell_format_left)
worksheet.merge_range(*merge_ranges[6], 'Date of joining in service',cell_format_left)
worksheet.merge_range(*merge_ranges[7], data[0][3].strftime('%d-%m-%Y'),cell_format_left)
worksheet.merge_range(*merge_ranges[8], 'Campus',cell_format_left)
worksheet.merge_range(*merge_ranges[9], 'ANNA UNIVERSITY REGIONAL CAMPUS   -   TIRUNELVELI',cell_format_center)
worksheet.merge_range(*merge_ranges[10],'DETAILS OF LEAVE', cell_format_left)
worksheet.merge_range(*merge_ranges[11],'Earned Leave  (EL)', cell_format_left)
worksheet.merge_range(*merge_ranges[12],'Medical Leave  (ML)  /  Maternity Leave  (MTL) / Loss of pay (LOP)', cell_format_left)
worksheet.write('L9', 'Type of Leave',cell_format_left)
if need_remark:
    worksheet.write('M10', 'Remarks',cell_format_left)

head_list = [['From', 'To', 'with permission on', 'Date of Joining on Duty after EL', 'Total No. of Days',
              'From', 'To', 'Total No. of Days', 'Medical Fitness on', 'with permission on',
              'Date of Joining on Duty after leave', '(ML/ EL/LOP/MTL)']
]

# Define the starting row
start_row = 9

# Iterate through each sublist and its values
for sublist in head_list:
    column_index = 0  # Start writing from the first column
    for value in sublist:
        worksheet.write(start_row, column_index, value,cell_format_left)
        column_index += 1  # Move to the next column for the next value
    start_row += 1 
    
start_row = 10   
for sublist in sorted_data:
     # Start writing from the first column
    if sublist[-1]=='el':
        column_index = 0 
        for value in sublist:
            if value =='el':
                worksheet.write(start_row, 11, value,cell_format_left)
            elif isinstance(value, datetime.date):
                worksheet.write(start_row, column_index, value.strftime('%d-%m-%Y'),cell_format_left)
                column_index += 1
            elif isinstance(value, str) and value.startswith("AURCT") and need_remark:
                worksheet.write(start_row, 12, value,cell_format_left)
            elif isinstance(value, list):
                
                if value:
                    worksheet.write(start_row, column_index,value[0] ,cell_format_left)
                else: 
                    worksheet.write(start_row, column_index,"Nil" ,cell_format_left)
                    
                column_index += 1
                
            elif isinstance(value, bytes):
                byte_data = value
                byte_data_str = byte_data.decode('utf-8')
                str_data=json.loads(byte_data_str)
                worksheet.write(start_row, column_index,str_data[0] ,cell_format_left)
                column_index += 1
            else:
                worksheet.write(start_row, column_index, value,cell_format_left)
                column_index += 1  # Move to the next column for the next value
        for i in range(column_index,11):
            worksheet.write(start_row, i,"-" ,cell_format_center)
            
        start_row += 1 
    else:
        column_index = 5
        for value in sublist:
            if value == 'lop' or value == 'mtl' or value == 'ml':
                print(value)
                worksheet.write(start_row, 11, value,cell_format_left)
            elif isinstance(value, datetime.date):
                worksheet.write(start_row, column_index, value.strftime('%d-%m-%Y'),cell_format_left)
                column_index += 1
                
            elif isinstance(value, str) and value.startswith("AURCT") and need_remark:
                worksheet.write(start_row, 12, value,cell_format_left)   
                
            elif isinstance(value, bytes):
                byte_data = value
                byte_data_str = byte_data.decode('utf-8')
                str_data=json.loads(byte_data_str)
                formatted_date = datetime.datetime.strptime(str_data[0], "%Y-%m-%d")
                worksheet.write(start_row, column_index,formatted_date.strftime('%d-%m-%Y') ,cell_format_left)
                column_index += 1
                
            elif isinstance(value, list):
                if value:
                    worksheet.write(start_row, column_index,value[0] ,cell_format_left)
                else:
                    worksheet.write(start_row, column_index,"Nil" ,cell_format_left)
                column_index += 1
                
                
            else:
                worksheet.write(start_row, column_index, value,cell_format_left)
                column_index += 1  # Move to the next column for the next value
        for i in range(0,5):
             worksheet.write(start_row, i,"-" ,cell_format_center)
            
        start_row += 1 
print(start_row)
worksheet.merge_range(*(start_row-1,0,start_row-1,12), 'sample last row'	, cell_format_center)   
workbook.close()

ml
ml
ml
20


In [104]:
import os
import openpyxl

# Open the workbook
workbook = openpyxl.load_workbook('book.xlsx')

# Access the worksheet
worksheet = workbook['Sheet1']

# Access a cell
cell = worksheet['A1']

# Print the value of the cell
print(cell.value)

# Close the workbook and delete it
workbook.close()
os.remove('book.xlsx')

Annexure -II


In [103]:
pip install openpyxl

     -------------------------------------- 250.0/250.0 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
key = ["yourname",
       "dest",
       "dep",
       "type",
       "reason",
       "dates"
       "leave_add",
       "notes",
       "date"
      ]

In [1]:
from docx import Document

yourname = input("Enter your name: ")
dest = input("Enter your destination: ")
dep = input("Enter your departure location: ")
type = input("Enter the type of travel: ")
reason = input("Enter the reason for travel: ")
dates = input("Enter the travel dates: ")
leave_add = input("Enter your leave address: ")
notes = input("Enter any notes: ")
date = input("Enter the current date: ")

doc = Document('lf.docx')

target_word = 'yourname'
replacement_text = yourname

# Iterate through paragraphs and runs to find and replace the target word
for paragraph in doc.paragraphs:
    for run in paragraph.runs:
        if target_word in run.text:
            new_text = run.text.replace(target_word, replacement_text)
            run.text = new_text

# Save the modified document
doc.save('modified_example.docx')

print("Word document 'modified_example.docx' with replaced word has been created.")


Word document 'modified_example.docx' with replaced word has been created.


In [5]:
from docx import Document

# Taking input from the user
yourname = input("Enter your name: ")
dest = input("Enter your destination: ")
dep = input("Enter your departure location: ")
travel_type = input("Enter the type of travel: ")
reason = input("Enter the reason for travel: ")
dates = input("Enter the travel dates: ")
leave_add = input("Enter your leave address: ")
notes = input("Enter any notes: ")
date = input("Enter the current date: ")

# Create a new document based on the template
doc = Document('lf.docx')

# Dictionary to store field names and their corresponding values
fields = {
    'yourname': yourname,
    'dest': dest,
    'dep': dep,
    'type': travel_type,
    'reason': reason,
    'dates': dates,
    'leave_add': leave_add,
    'notes': notes,
    'date': date
}

# Function to replace placeholders in the document
def replace_placeholder(doc, target, replacement):
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if target in run.text:
                new_text = run.text.replace(target, replacement)
                run.text = new_text

# Iterate through fields and replace placeholders with values
for field, value in fields.items():
    replace_placeholder(doc, field, value)

# Save the modified document
doc.save('filled_document.docx')

print("Document filled and saved as 'filled_document.docx'")

#timepass
import subprocess
import os
subprocess.Popen(["filled_document.docx"], shell=True)
input("Press Enter after you've closed the Excel file...")
os.remove("filled_document.docx")

Enter your name:  maheshwar
Enter your destination:  master
Enter your departure location:  do not know
Enter the type of travel:  space
Enter the reason for travel:  timepass
Enter the travel dates:  12/7/1993
Enter your leave address:  earth
Enter any notes:  o
Enter the current date:  12/8/2019


Document filled and saved as 'filled_document.docx'


Press Enter after you've closed the Excel file... 1


In [ ]:
cursor.reset()
query = "SELECT `from`, `to`, `with_permission_on`, `date_of_rejoin`, `total` FROM `el` WHERE id="+id+"; "
cursor.execute(query)
data2 = cursor.fetchall()
data2
data_el = [sublist + ("el",) for sublist in data2]

cursor.reset()
query = "SELECT `from`, `to`, `medical_fittness_on`, `with_permission_on`, `doj`, `total` FROM `lop` WHERE id="+id+"; "
cursor.execute(query)
data3 = cursor.fetchall()
data_lop = [sublist + ("lop",) for sublist in data3]

cursor.reset()
query = "SELECT `from`, `to`, `medical_fittness_on`, `with_permission_on`, `doj`, `total` FROM `ml` WHERE id="+id+"; "
cursor.execute(query)
data4 = cursor.fetchall()
data_ml = [sublist + ("ml",) for sublist in data4]

cursor.reset()
query = "SELECT `from`, `to`, `medical_fittness_on`, `with_permission_on`, `doj`, `total` FROM `mtl` WHERE id="+id+"; "
cursor.execute(query)
data5 = cursor.fetchall()
data_mtl = [sublist + ("mtl",) for sublist in data5]


In [80]:
def getdata(type, id):
    if type=='el':
        cursor.reset()
        query = "SELECT `from`, `to`, `from_prefix`, `to_prefix`, `from_suffix`, `to_suffix`, `date_of_rejoin`, `total` FROM "+type+" WHERE id="+id+"; "
        cursor.execute(query)
        data_final = cursor.fetchall()
        
        on_permission = []
        for i in data_final:
            on_permission.append(with_permission_fun(i[2],i[3],i[4],i[5]))
        remark = []
        for i in data_final:
            print(data[0][2],i[0].year)
            remark.append(attendence_regsiter(data[0][2],i[0].year))
        remark = [item[0] for sublist in remark for item in sublist]
            
        data_final_updated = []
        for i,sublist in enumerate(data_final):
            columns_to_keep = sublist[:2] + (on_permission[i],) + sublist[6:] + (remark[i],)
            data_final_updated.append(columns_to_keep)
        data_final_updated = [sublist + (type,) for sublist in data_final_updated]
        return data_final_updated
    else:
        cursor.reset()
        query = "SELECT `from`,`to`,`total`, `medical_fittness_on`, `from_prefix`, `to_prefix`, `from_suffix`, `to_suffix`, `doj` FROM "+type+" WHERE id="+id+";; "
        cursor.execute(query)
        data_final = cursor.fetchall()
        on_permission = []
        for i in data_final:
            on_permission.append(with_permission_fun(i[4],i[5],i[6],i[7]))
        remark = []
        for i in data_final:
            print(data[0][2],i[0].year)
            remark.append(attendence_regsiter(data[0][2],i[0].year))
        remark = [item[0] for sublist in remark for item in sublist]
        data_final_updated = []
        for i,sublist in enumerate(data_final):
            columns_to_keep = sublist[:4] + (on_permission[i],) + sublist[8:] + (remark[i],)
            data_final_updated.append(columns_to_keep)
        data_final_updated = [sublist + (type,) for sublist in data_final_updated]
        return data_final_updated
            
        
    

In [81]:
data_el = getdata('el','22015')
data_ml = getdata('ml','22015')
data_lop = getdata('lop','22015')
data_mtl = getdata('mtl','22015')

CSE 2015
CSE 2015
CSE 2015
CSE 2015
CSE 2020
CSE 2020
CSE 2020
CSE 2022
CSE 2022
CSE 2022


In [94]:
data_ml = getdata('el','22015')

In [70]:
data_el

[(datetime.date(2015, 5, 4),
  datetime.date(2015, 5, 8),
  ['01-05-2015 to 03-05-2015 & 09-05-2015 to 10-05-2015'],
  datetime.date(2015, 5, 11),
  5,
  'AURCT / AR / 2015/ 03 of 13',
  'el'),
 (datetime.date(2015, 7, 7),
  datetime.date(2015, 7, 11),
  ['12-07-2015'],
  datetime.date(2015, 7, 13),
  5,
  'AURCT / AR / 2015/ 03 of 13',
  'el'),
 (datetime.date(2015, 8, 10),
  datetime.date(2015, 8, 14),
  ['08-08-2015 to 09-08-2015 & 15-08-2015 to 16-08-2015'],
  datetime.date(2015, 8, 17),
  5,
  'AURCT / AR / 2015/ 03 of 13',
  'el'),
 (datetime.date(2015, 9, 28),
  datetime.date(2015, 10, 1),
  ['27-09-2015 & 02-10-2015 to 04-10-2015'],
  datetime.date(2015, 10, 5),
  4,
  'AURCT / AR / 2015/ 03 of 13',
  'el'),
 (datetime.date(2020, 3, 11),
  datetime.date(2020, 3, 14),
  [],
  None,
  4,
  'AURCT / AR / 2020 / 03 of 18 ',
  'el'),
 (datetime.date(2020, 5, 19),
  datetime.date(2020, 5, 23),
  [],
  None,
  5,
  'AURCT / AR / 2020 / 03 of 18 ',
  'el'),
 (datetime.date(2020, 11, 5)

In [82]:
sorted_data = sorted(data_lop+data_el+data_ml+data_mtl, key=lambda x: x[0])
for item in sorted_data:
    print(item)

(datetime.date(2015, 5, 4), datetime.date(2015, 5, 8), ['01-05-2015 to 03-05-2015 & 09-05-2015 to 10-05-2015'], datetime.date(2015, 5, 11), 5, 'AURCT / AR / 2015/ 03 of 13', 'el')
(datetime.date(2015, 7, 7), datetime.date(2015, 7, 11), ['12-07-2015'], datetime.date(2015, 7, 13), 5, 'AURCT / AR / 2015/ 03 of 13', 'el')
(datetime.date(2015, 8, 10), datetime.date(2015, 8, 14), ['08-08-2015 to 09-08-2015 & 15-08-2015 to 16-08-2015'], datetime.date(2015, 8, 17), 5, 'AURCT / AR / 2015/ 03 of 13', 'el')
(datetime.date(2015, 9, 28), datetime.date(2015, 10, 1), ['27-09-2015 & 02-10-2015 to 04-10-2015'], datetime.date(2015, 10, 5), 4, 'AURCT / AR / 2015/ 03 of 13', 'el')
(datetime.date(2020, 3, 11), datetime.date(2020, 3, 14), [], None, 4, 'AURCT / AR / 2020 / 03 of 18 ', 'el')
(datetime.date(2020, 5, 19), datetime.date(2020, 5, 23), [], None, 5, 'AURCT / AR / 2020 / 03 of 18 ', 'el')
(datetime.date(2020, 11, 5), datetime.date(2020, 11, 11), [], datetime.date(2020, 11, 12), 7, 'AURCT / AR / 2020

In [21]:
data_ml[0][0].year

2022

In [30]:
def attendence_regsiter(dep,year):
    query = "SELECT register_name FROM `attendance_register` WHERE department = %s AND year = %s;"
    cursor.execute(query, (dep, year))
    data = cursor.fetchall()
    return data

In [41]:
remark = []
for i in sorted_data:
    remark.append(attendence_regsiter(data[0][2],i[0].year))

In [42]:
remark = [list(item[0]) for item in remark]

In [44]:
remark[0]

['AURCT / AR / 2015/ 03 of 13']

In [45]:
data[0][2]

'CSE'

In [61]:
data_el[-1][-1]

['AURCT / AR / 2020 / 03 of 18 ']